In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from deap import base, creator, tools, algorithms
import random
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pygad
import torch


ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: initialization failed

In [ ]:
file_path = 'dataset/cardio_train.csv'

df = pd.read_csv(file_path, sep=';')

df.head()

In [ ]:
df.info()

In [ ]:
columns = df.columns.drop('cardio')
print(columns)

In [ ]:
# replace age in days to age in years

df['age'] = df['age'] / 365
df['age'] = df['age'].astype(int)

df.head()

In [ ]:
df.describe()

In [ ]:
df['cardio'].value_counts()

In [ ]:
df['cardio'].value_counts().plot.pie(autopct='%1.1f%%')


In [ ]:
# violin plot for age and cardio

plt.figure(figsize=(10, 6))

sns.violinplot(x='cardio', y='age', data=df)

plt.title('Violin plot for age and cardio')

plt.show()

In [ ]:
# violin plot for with and cardio

plt.figure(figsize=(10, 6))

sns.violinplot(x='cardio', y='weight', data=df)

plt.title('Violin plot for weight and cardio')

plt.show()


In [ ]:
# count of male and female cardio patients

plt.figure(figsize=(10, 6))
sns.countplot(x='gender', hue='cardio', data=df)

In [ ]:
# count of non-smokers and smokers cardio patients

plt.figure(figsize=(10, 6))

sns.countplot(x='smoke', hue='cardio', data=df)

In [ ]:
# count of non-alcoholics and alcoholics cardio patients

plt.figure(figsize=(10, 6))

sns.countplot(x='alco', hue='cardio', data=df)

In [ ]:
# count of physically active and inactive cardio patients

plt.figure(figsize=(10, 6))

sns.countplot(x='active', hue='cardio', data=df)


In [ ]:
# count of Gluc0, Gluc1, Gluc2 cardio patients

# gluc: 1 - normal, 2 - above normal, 3 - well above normal

plt.figure(figsize=(10, 6))

sns.countplot(x='gluc', hue='cardio', data=df)

In [ ]:
# count of Cholesterol cardio patiens

# cholesterol: 1 - normal, 2 - above normal, 3 - well above normal

plt.figure(figsize=(10, 6))

sns.countplot(x='cholesterol', hue='cardio', data=df)

In [ ]:
# plot distribution of age

plt.figure(figsize=(10, 6))

sns.histplot(df['age'], bins=100, kde=True)

plt.title('Distribution of age')

plt.show()

In [ ]:
# plot distribution of weight

plt.figure(figsize=(10, 6))

sns.histplot(df['weight'], bins=100, kde=True)

plt.title('Distribution of weight')

plt.show()

In [ ]:
# conut of alcoholics and non-alcoholics Cholesterol patients

plt.figure(figsize=(10, 6))


sns.countplot(x='alco', hue='cholesterol', data=df)

plt.title('Count of alcoholics and non-alcoholics Cholesterol patients')

plt.show()

In [ ]:
# conut of alcoholics and non-alcoholics Gluc patients

plt.figure(figsize=(10, 6))

sns.countplot(x='alco', hue='gluc', data=df)

plt.title('Count of alcoholics and non-alcoholics Gluc patients')

In [ ]:
# count of smokers and non-smokers Cholesterol patients

plt.figure(figsize=(10, 6))

sns.countplot(x='smoke', hue='cholesterol', data=df)

plt.title('Count of smokers and non-smokers Cholesterol patients')

plt.show()

In [ ]:
# count of smokers and non-smokers Gluc patients

plt.figure(figsize=(10, 6))

sns.countplot(x='smoke', hue='gluc', data=df)

plt.title('Count of smokers and non-smokers Gluc patients')

In [ ]:
# group by blood pressure
# 1- Normal (sys <= 120 and dia <= 80)
# 2- at risk (120 < sys <= 140 or 80 < dia <= 90)
# 3- high (sys > 140 or dia > 90)

df['blood_pressure'] = 0

df.loc[(df['ap_hi'] <= 120) & (df['ap_lo'] <= 80), 'blood_pressure'] = 1

df.loc[((df['ap_hi'] > 120) & (df['ap_hi'] <= 140)) | ((df['ap_lo'] > 80) & (df['ap_lo'] <= 90)), 'blood_pressure'] = 2

df.loc[(df['ap_hi'] > 140) | (df['ap_lo'] > 90), 'blood_pressure'] = 3

df.head()


In [ ]:
# check count of cardio patients by blood pressure

plt.figure(figsize=(10, 6))

sns.countplot(x='blood_pressure', hue='cardio', data=df)

plt.title('Count of cardio patients by blood pressure')

plt.show()

In [ ]:
# check count of smokers and non-smokers by blood pressure

plt.figure(figsize=(10, 6))

sns.countplot(x='blood_pressure', hue='smoke', data=df)

plt.title('Count of smokers and non-smokers by blood pressure')

plt.show()

In [ ]:
# check count of alcoholics and non-alcoholics by blood pressure

plt.figure(figsize=(10, 6))

sns.countplot(x='blood_pressure', hue='alco', data=df)

plt.title('Count of alcoholics and non-alcoholics by blood pressure')

In [ ]:
# check effect of blood pressure on cholesterol

plt.figure(figsize=(10, 6))

sns.countplot(x='blood_pressure', hue='cholesterol', data=df)

plt.title('Effect of blood pressure on cholesterol')

plt.show()

In [ ]:
# check effect of weight on blood pressure

plt.figure(figsize=(10, 6))

sns.violinplot(x='blood_pressure', y='weight', data=df)

plt.title('Effect of weight on blood pressure')

plt.show()

In [ ]:
# distribution of blood pressure by age

plt.figure(figsize=(10, 6))

sns.violinplot(x='blood_pressure', y='age', data=df)

plt.title('Distribution of blood pressure by age')

plt.show()

In [ ]:
# calculate BMI

df['bmi'] = df['weight'] / (df['height'] / 100) ** 2

df.head()

In [ ]:
# violin plot for BMI and cardio

plt.figure(figsize=(10, 6))

sns.violinplot(x='cardio', y='bmi', data=df)

plt.title('Violin plot for BMI and cardio')

plt.show()

In [ ]:
# violin plot for BMI and blood pressure

plt.figure(figsize=(10, 6))

sns.violinplot(x='blood_pressure', y='bmi', data=df)

plt.title('Violin plot for BMI and blood pressure')

plt.show()

In [ ]:
# correlation matrix

plt.figure(figsize=(12, 8))

sns.heatmap(df.corr(), annot=True)

plt.title('Correlation matrix')

plt.show()

In [34]:
df.fillna(df.median(), inplace=True)
df = df.drop(columns=['id'],axis=1)

# Extract features (all columns except 'cardio') and target variable ('cardio')
X = df.drop(columns=['cardio'],axis=1)
y = df['cardio']

columns_to_standardize = ['age', 'height','weight','ap_hi','ap_lo','bmi']


# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

# Standardize the feature values
scaler = StandardScaler()
X_train[columns_to_standardize] = scaler.fit_transform(X_train[columns_to_standardize])
X_test[columns_to_standardize] = scaler.fit_transform(X_test[columns_to_standardize])
X_val[columns_to_standardize] = scaler.fit_transform(X_val[columns_to_standardize])

In [35]:
# Create the Fitness and Individual classes
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def evaluate(individual):
    feature_mask = individual[:num_features]
    hyperparameters = individual[num_features:]
    
    X_train_selected = X_train[:, feature_mask == 1]
    X_val_selected = X_val[:, feature_mask == 1]

    num_hidden_layers = int(individual[-3])
  
    # Extract the sizes of the hidden layers 
    hidden_layers = tuple(hyperparameters[:num_hidden_layers])
  
    # Extract other hyperparameters
    activation = ['identity', 'logistic', 'tanh', 'relu'][individual[-2]]
    solver = 'adam'
    alpha = individual[-1]
    # Initialize the MLPClassifier with the hyperparameters
    model = MLPClassifier(hidden_layer_sizes=hidden_layers, activation=activation,
                          solver=solver, alpha=alpha, max_iter= 1000, random_state=42)
  
    # Train the model
    model.fit(X_train, y_train)
  
     # Predict on the validation set
    y_pred = model.predict(X_val)
   
    # Calculate accuracy
    accuracy = accuracy_score(y_val, y_pred)
   
     # Return the fitness score (accuracy)
    return accuracy,


 # Define the genetic algorithm components
toolbox = base.Toolbox()


# Define individual components: up to 20 hidden layers, activation, solver, and alpha
num_features = X_train.shape[1]
toolbox.register("attar_feature_mask", random.randint, 0, 1)
toolbox.register("attr_hidden_layer_size", random.randint, 10, 200)
toolbox.register("attr_num_hidden_layers", random.randint, 1, 10)  # 1 to 10 hidden layers
toolbox.register("attr_activation", random.randint, 0, 3)
toolbox.register("attr_alpha", random.uniform, 0.0001, 0.01)

# Define the individual and population
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_hidden_layer_size,
                  toolbox.attr_hidden_layer_size,
                  toolbox.attr_hidden_layer_size,
                  toolbox.attr_hidden_layer_size,
                  toolbox.attr_hidden_layer_size,
                  toolbox.attr_hidden_layer_size,
                  toolbox.attr_hidden_layer_size,
                  toolbox.attr_hidden_layer_size,
                  toolbox.attr_hidden_layer_size,
                  toolbox.attr_hidden_layer_size,
                  toolbox.attr_num_hidden_layers,
                  toolbox.attr_activation,
                  toolbox.attr_alpha), n=1)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate",tools.cxTwoPoint)

# Update mutation function to ensure valid hidden layer sizes
toolbox.register("mutate", tools.mutUniformInt, 
                 low[0] * num_features + [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0.0001],  #minimum size of 1 for hidden layers
                 up[1] * num_features + [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 10, 3, 2, 0.01], indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Main function to run the genetic algorithm
def main():
    random.seed(42)
  
    # Initialize population
    population = toolbox.population(n=40)
  
    # Apply the genetic algorithm
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, verbose=True)
  
    # Get the best individual
    best_individual = tools.selBest(population, k=1)[0]
    print("Best individual is:", best_individual)
    print("Best validation accuracy is:", evaluate(best_individual)[0])

main()